# L1 Language Models, the Chat Format and Tokens

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### helper function
This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

## Prompt the model and get a completion

In [4]:
response = get_completion("What is the capital of France?")

In [5]:
print(response)

The capital of France is Paris.


## Tokens

In [6]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

polilol


"lollipop" in reverse should be "popillol". LLM could not understand that word are made up from a sequences of alphabets. That's because words and sentences are made up from "tokens" from the [tokenization](https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html) process in NLP, as shown in each colour breakdown in the diagram below.

<div><img src="images/llm_tokens.png" style="width: 50%"/></div>

 A trick is to use delimiters to force the tokenzier to treat each alphabet as a token.
<div><img src="images/llm_tokens_2.png" style="width: 50%"/></div>

In [7]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")
print(response)

p-o-p-i-l-l-o-l


## Helper function (chat format)
Here's the helper function we'll use in this course.

In [8]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

<div><img src="images/llm_system_role.png" style="width: 50%"/></div>

In [9]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, look at this carrot so happy and bright,
It fills me with joy, such a glorious sight.
Its orange hue shines so perfectly true,
Such a happy little veggie, just like me and you!


In [10]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, there was a happy carrot named Carl who loved nothing more than soaking up the sun and growing tall and plump in his cozy garden patch.


In [11]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

There once was a carrot, so full and so bright, who spent all his days in the sun's warm light, he grew and he grew, like a tower so tall, spreading his joy to one and to all.


In [12]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [13]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [14]:
print(response)
print(token_dict)

Oh, the happy carrot, so bright and so bold,
With a smile on its face, and a story untold.
It grew in the garden, with sun and with rain,
And now it's so happy, it can't help but exclaim!
{'prompt_tokens': 39, 'completion_tokens': 52, 'total_tokens': 91}


#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

<div><img src="images/llm_api_key.png" style="width: 50%"/></div>

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.

<div><img src="images/llm_prompt_based_ai.png" style="width: 50%"/></div>

At current state of prompt-based AI, it is very good at handling unstructured data(news articles, sequence of texts) vs. structured data(e.g. data in tables, lists, spreadsheets).